<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>


# <font color="#76b900">**4:** 用于序列生成的编码器和解码器</font>

在之前的 notebook 中，我们探讨了“BERT 类”仅编码模型在理解静态输入方面的优势。这些模型在文本分类、情感分析和命名实体识别等不需要转换或动态改变输入的任务中表现出色。然而，它们生成新颖或动态输出的能力有限。

对于机器翻译、摘要和问答等任务，输出需要以结构化的形式动态生成，我们需要比对输入进行编码更复杂的架构。

在这个 notebook 中，我们扩展我们的架构工具包，来探索**编码器-解码器**和**仅解码器**模型，这些模型旨在根据输入上下文生成有序的序列：

- **编码器-解码器模型（例如，Flan-T5）**：这些模型首先将输入编码为固定表示，然后再解码为新的有序序列。例如，在翻译中，编码器理解原句子，而解码器生成翻译后的句子。
- **仅解码器模型（例如，GPT-2）**：与编码器-解码器模型不同，仅解码器模型基于之前的上下文预测 token，并在输入和输出分布相交叠的任务中表现出色，比如开放式文本生成和对话系统。

#### **学习目标:**

- 了解使用编码器处理静态上下文并使用解码器生成有序序列的编码器-解码器模型。
- 理解像 GPT-2 这样的仅解码器模型如何通过仅基于之前解码器上下文预测 token，在生成任务中表现出色。


<hr>
<br>

## **4.1:** 机器翻译任务

[**机器翻译**](https://huggingface.co/tasks/translation)是指使用软件自动将文本从一种语言翻译成另一种语言的任务。虽然这个术语听起来很简单，但机器翻译是一个极其复杂的任务，需要模型理解和生成语法、句法、词序甚至文化细微差别截然不同的语言。

例如，从一种像日语这样的语言翻译到英语，日语通常将动词放在句子的末尾，而英语遵循主谓宾（SVO）结构，这可能会很棘手。此外，语言通常有独特的习惯用语和上下文含义，这些必须被模型准确捕捉。

### **从编码器转向解码器**
翻译的复杂性不仅在于理解源语言（编码），还在于生成流畅且连贯的目标语言翻译（解码）。这就是**编码器-解码器模型** 挥作用的地方。特别是在自然语言方面：

- **编码器**处理源语言句子，将其转化为一个固定的表示，捕捉其含义。
- **解码器**采用这个表示，逐字生成目标语言的翻译句子。

这种组合让模型在尝试“生成”翻译之前，首先“理解”句子的意思。这样的结构化方法可以导致更准确的翻译，尤其是在句子结构截然不同的语言中。

然而，对于其他生成任务，比如创建开放式文本或对话，我们可以绕过固定输入表示的需要，而依赖于一种逐个 token 生成输出的模型。这将引导我们进入下一部分，探索**仅解码器**模型，如 GPT-2，它们专为这些更开放式的任务设计。

<hr>
<br>

## **4.2:** 引入 GPT 风格模型

正如我们所讨论的，像 BERT 这样的架构在理解输入数据方面表现出色，但在生成文本方面却有所不足。对于需要生成新序列的任务——无论是讲故事、对话还是开放式文本补全——我们需要一个能够逐个预测和生成 token 的模型。这时**GPT 风格的架构**就展现出它的优势了。

#### **为什么使用仅解码器模型？**
GPT-2 是一个**仅解码器**模型的例子，专为**自回归文本生成**设计。与使用单独编码器来理解输入的编码器-解码器模型不同，仅解码器模型是逐个 token 生成文本。该模型的训练目标是根据前面的 token 来预测序列中的下一个 token，这个过程称为**自回归**。

#### **自回归生成是如何工作的？**
1. 模型接收一个初始提示词，比如 `"Hello world,"`。
2. 它根据提示词预测最可能的下一个 token。
3. 预测的 token 被添加到输入序列中，重复这个过程，直到模型生成完整的输出。

这种方法使得模型能够生成开放式序列，而不需要固定的输入上下文，非常适合对话生成、讲故事和文本补全等任务。

**让我们通过一个简单的文本生成示例来看看 GPT-2 的实际效果：**

In [1]:
from transformers import pipeline, set_seed

# Create a text-generation pipeline using GPT-2
generator = pipeline('text-generation', model='gpt2')

# Generate 5 sequences of text starting with the prompt "Hello world"
generator("Hello world,", max_length=20, num_return_sequences=5)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'generated_text': "Hello world, my dear boy! Come back from the dead as best you can, for I'm"},
 {'generated_text': "Hello world, I have nothing new to say.\n\nYou'll need an HTML5 capable browser"},
 {'generated_text': "Hello world, it's all about getting a life of yourself. You know your name and not your"},
 {'generated_text': "Hello world, it's like we're trying to break the world for a little while and we'll"},
 {'generated_text': 'Hello world, why can\'t you do better to this enemy?!" At this, the two men\'s'}]


#### **解读 GPT-2 输出**
当模型生成文本时，它是基于提示词 "Hello world," 逐个预测 token 来创造新内容的。由于模型的概率性质，每个序列都会有所不同。

- **为什么是 5 个序列？**: 通过指定 `num_return_sequences=5`，我们要求模型生成 5 个文本变体。GPT 风格模型使用 `temperature` 和 `top_k`/`top_p` 等属性来调整采样策略，默认情况下结果是非确定性的。
- **最大长度**: 参数 `max_length=20` 将输出限制为 20 个 token，这有助于在实验时管理生成序列的大小。在此之前，模型也可以通过专门的停止 token 或手动指定的停止字符串来停止生成。

#### **探究前向传播**

在工作流的更深层次上，我们可以运用之前相同的原则，探讨数据实际上是如何生成的：

In [2]:
import torch

print("GENERATING ALL AT ONCE:")
input_str = "Hello world"
print(f"{(x := generator.preprocess(input_str))}")
print(f"{(x := generator.forward(x, max_new_tokens=20))}")
print(f"{(x := generator.postprocess(x))}")

#################################################################################

print("\nGENERATING ONE TOKEN AT A TIME (prep+forward+post):")
print(input_str := "Hello world", end="")
output_buffer = ""
for i in range(20):
    x = generator.preprocess(input_str + output_buffer)
    x = generator.forward(x, max_new_tokens=1)
    x = generator.postprocess(x)
    next_word = x[0].get("generated_text")[len(input_str + output_buffer):]
    output_buffer += next_word
    print(next_word.replace("\n", "\\n"), end="")

#################################################################################

print("\n\nGENERATING ONE TOKEN AT A TIME (manually, greedily-sampled):")

model_body = generator.model.transformer
model_head = generator.model.lm_head
tknzr_encode = generator.tokenizer.encode
tknzr_decode = generator.tokenizer.decode

def compute_embed(token_id):
    return model_body.wte(torch.tensor([token_id])).view(1, -1, 768)

# PREFILL stage: Processing the initial input string
print(input_str := "<|endoftext|> Hello world", end="")
embed_buffer = compute_embed(tknzr_encode(input_str))
attention_mask = torch.ones(embed_buffer.shape[:2], dtype=torch.long)
past_key_values = None

# PREFILL - running the model for the input string, getting kv cache and embeddings
prefill_output = model_body.forward(
    inputs_embeds=embed_buffer, 
    attention_mask=attention_mask,
    past_key_values=past_key_values,
)
past_key_values = prefill_output.get("past_key_values")
predicted_embed = prefill_output.get("last_hidden_state")

# DECODE stage: Start the token-by-token generation process
for i in range(100):
    predicted_probs = model_head(predicted_embed[:, -1, :])
    predicted_token = torch.argmax(predicted_probs, dim=-1).item()
    print(tknzr_decode(predicted_token), end="")

    # Update attention mask and run model with past_key_values for next token
    decode_output = model_body.forward(
        inputs_embeds=compute_embed(predicted_token), 
        attention_mask=torch.ones([1,1], dtype=torch.long),
        past_key_values=past_key_values,
    )
    predicted_embed = decode_output.get("last_hidden_state")
    past_key_values = decode_output.get("past_key_values")

GENERATING ALL AT ONCE:
{'input_ids': tensor([[15496,   995]]), 'attention_mask': tensor([[1, 1]]), 'prompt_text': 'Hello world'}
{'generated_sequence': tensor([[[15496,   995,   318,   616,  2988,   338,    13,  2011,  2988,   531,
            284,   502,    11,   366,  5211,   345,   760,   611,   314,  1833,
            393,   407]]]), 'input_ids': tensor([[15496,   995]]), 'prompt_text': 'Hello world'}
[{'generated_text': 'Hello world is my father\'s. My father said to me, "Do you know if I understand or not'}]

GENERATING ONE TOKEN AT A TIME (prep+forward+post):
Hello world, people are dying. We didn't want to get killed by another person."\n\n\n(

GENERATING ONE TOKEN AT A TIME (manually, greedily-sampled):
<|endoftext|> Hello world!

I'm a programmer and I'm a big fan of the Java programming language. I've been using Java since I was a kid and I've been using it for a long time. I've been using Java for a long time and I've been using it for a long time. I've been using Java for

<br>

看看怎么运作的，*似乎*这个过程可以通过一个简单的 $n \to 1$ 编码器模型重复执行，直到达到最大长度或停止 token。您大致是对的，*只不过有一个问题*：**训练效率低。**

给定一个训练示例 `"Hello world and all who live in it"`，一个 $n \to 1$ 的训练形式将需要以下训练示例：
```sh
"<CLS> Hello" -> "world"
"<CLS> Hello world" -> "and"
"<CLS> Hello world and" -> "all"
"<CLS> Hello world and all" -> "who"
"<CLS> Hello world and all who" -> "live"
"<CLS> Hello world and all who live" -> "in"
"<CLS> Hello world and all who live in" -> "it"
"<CLS> Hello world and all who live in it" -> "<PAD>"
```  

假设我们有一个常规的**双向推理**编码器结构，每生成一个新词，注意力都必须完全重新计算。相反，如果我们的注意力机制仅为**单向**，那么我们可以从一个限制的注意力矩阵得到以下训练形式：

```sh
INPUT="<s> Hello world and all who live in it"
         \    -\   -\   -\  -\  -\  -\  -\ -\
OUTPUT="Hello world and all who live in  it </s>"
```


**简化直觉：** 一般来说，当输入序列逐步增长时，单向推理更好，因为每个输入-输出映射可以并行训练整个序列。双向推理更适合当输入序列不增长时，因为每个预测可以考虑它之前和之后的条目。

在常见的术语中，这个属性——或者说，正是需要这个属性的下一个 token 预测任务——是**编码器**和**解码器** transformer 架构之间的区别。 
> <div><img src="imgs/bert-vs-gpt.png" width="600"/></div>
>
> **来源: [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (2019)](https://arxiv.org/abs/1810.04805)**-


---

<details>
<summary><b>数学细节：</b></summary>

当设置一个自回归问题，目标是从一些条件序列 $x_{0 \ldots a-1}$ 生成一些新的序列 $x_{a\ldots b}$ 时，我们会在每个时间步生成 $x_t$，条件是 $a \leq t \leq b$。因此：

- 在单向的情况下，我们建模 $P(x_{a\ldots t} \ | \ x_{0\ldots t-1})$ 从某个起始点 $a$ 开始，保证 $P(x_{s} \ | \ x_{0\ldots s-1})$ 对所有 $s \leq t$，所以之前的预测 $P(x_s)$ 不需要为每个 $t$ 增量重新计算。当 $t$ 增长时，这种方式更好，因为每个过去的生成 $P(x_s)$ 有一个相对于 $s$ 的固定定义。

- 在双向推理中，我们建模 $P(x_{a\ldots t} \ | \ x_{0\ldots t-1})$ 使得 $P(x_s \ | \ x_{0\ldots t-1})$ 对所有 $s \leq t$，这会使每一个 $P(x_s)$ 同时依赖于过去和未来的 token 预测。当 $t$ 静态时，这种方式更好，因为每个 $P(x)$ 有更多的信息进行条件化，但也有一个静态定义，不会随着更多条目的引入而变化/需要重新计算。

**注意：** 当输出范围实际上是不同的分布时，双向推理更具吸引力——比如我们构造 $P(y | x)$ 而不是 $P(x_b | x_a)$，用于不重叠的 $x$ 和 $y$ 分布——因为我们的网络可以复用这种直觉，并从增加的训练中受益。可以想想自然语言翻译任务，您只想得到输入子串的翻译，而不需要其它部分。

</details>

<hr>
<br>

## **4.3：** 编码器、解码器和编码-解码器

之前我们将编码器和解码器简化为自然语言的应用，所以这一部分希望更一般地定义这些术语。在任何机器学习的框架中，您会处理以下类型的数据表示：
- **显式（观察到的）表示：** 人类/软件可以解释的数据形式。
    - 也就是说，输入模型的实际语言/编码语言/数据点等。
- **隐式（潜在的）表示：** 为实现最终目标而优化出来的数据形式。
    - 也就是说，多层工作流中的中间表示、为相似性搜索优化的嵌入等。

从这个角度看，您通常会在深度学习工作流中与两种宏观结构交互：
- **编码器：** 将输入转换为具有期望属性（即维度、语义、范围等）的某种隐式表示。
- **解码器：** 将输入直接转换为某种显式表示（即人类/软件合理的数据格式）。

那么，为什么 BERT 被认为是一种编码器架构呢？原因有点历史性，归结为以下直觉：
- **如果我们的任务是创建一个简单的 $n \to n$ 或 $n \to 1$ 映射，** 之前的 BERT 类架构就足够了。可以说，基于 BERT 的工作流具有一个逐 token 的 MLP，充当一个 ***逐 token/逐序列解码器***。但这可能令人困惑，因此我们之前提到的特定任务头或分类头更为常见。
- **如果我们的任务是逐步生成一个新的 $m$-序列，** 这种架构是不够的，需要一个二级的 ***文本生成解码器*** 结构，帮助以不同方式建模我们的数据。

一般来说，专用解码器组件具有更适合生成显式输出表示的某种属性。这可以是一个序列、图像、图表、物理约束的系统等。***在这种情况下，解码器的作用更适合于逐 token 的 *自回归* 输出生成。***

在原始的 [**“Attention Is All You Need”（2017 年论文）**](https://arxiv.org/abs/1706.03762) 中，这两个结构发挥了各自的优势，并结合在一起形成了“条件解码”，利用一个序列的双向推理来帮助生成另一个序列，采用单向逐 token 生成。它们使用的策略称为 **交叉注意力**，即同时考虑两个序列的组件。

**最终结果是一个具有两个关键功能的架构：**
- 从解码器架构自回归地生成一个个 token，每个新生成的 token 都包含在预测下一个 token 的输入中。
- 经常将上下文从编码器注入到解码器中，确保生成与整体目标保持一致。

---

<details>
<summary><strong>数学细节：</strong></summary>

我们可以证明，之前的 notebook 中的注意力机制可以用于同时处理 $n$-元素和 $m$-元素序列，只要我们正确选择输入。考虑一种情况：您有来自编码器的查询/值 $K_{1..m}$/$V_{1..m}$ 和来自解码器的键 $Q_{1..n}$。

> 
- 如果 $K_i$ 和 $Q_i$ 具有相同的嵌入维度，那么 $Q_iK_i^T$ 是一个 $n\times m$ 矩阵，这个矩阵的 softmax 值也是如此。换句话说：
 $$\text{Attention}(K_{1..m}, Q_{1..n}) \text{ 是 } n\times m.$$

- 由于 $V_{1..m}$ 是 $m \times d$ 的维度，因此与 $n\times m$ 注意力矩阵是可以相乘的：
 $$\text{Attention}(K_{1..m}, Q_{1..n}) V_{1..m} \text{ 是 } n\times d$$

- 由于 $\text{Attention}(K_{1..m}, Q_{1..n}) V_{1..m}$ 和 $\text{Attention}(K_{1..n}, Q_{1..n}) V_{1..n}$ 具有相同的维度，二者可以互换使用或串联使用。

因此，我们可以使用注意力接口将 $m$-元素序列作为 $n$-元素序列的上下文！只需多次重复这个过程，就能实现强大的上下文驱动生成。
</details>

<hr>
<br>

## **4.4：** 使用 T5 风格的编码器-解码器进行机器翻译

自从 2017 年的原始论文以来，我们能看到编码器-解码器架构在各个应用中有不同侧重。**那些仍然以编码器-解码器为主的用例通常满足以下标准：**

1. **需要从至少两个潜在不同长度的序列进行推理。**
    - 如果它们长度相同或者输出是严格的子集，那么使用编码器进行 $n \to n$ 映射就足够了。
2. **需要逐步生成或继续至少一个序列。**
    - 如果输入序列不增长，那么使用编码器进行 $n \to 1$ 映射就足够了。
3. **序列遵循不相交（disjoint）的分布（格式、目的、模态等）。**
    - 如果它们遵循相同的分布，更好的是将它们同时通过同一网络路径处理。
4. **需要是轻量且任务特定的，以便进行训练和/或提高性能。**
    - 如果模型可以是通用的/多用途的，我们可以将两个序列都通过解码器路径处理。*稍后会详细讨论。*

巧合的是，高速机器翻译仍然是这样的一个应用：

In [3]:
from transformers import pipeline

translator = pipeline('translation_en_to_fr', model='t5-base', device="cuda")
translator(["Hello World! How's it going?", "What's your name?"])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[{'translation_text': 'Bonjour Monde, comment se passe-t-il ?'},
 {'translation_text': 'Quel est votre nom?'}]

注意到这个任务满足我们所有的要求：
- 一段文字的翻译既不是固有的 $n \to n$ 也不是 $n \to 1$。
- 输出需要逐个 token 生成，同时保持连贯性。
- 这两种语言很可能遵循根本不同的分布。
- 模型应该快速、轻量，并且目的明确，灵活性是次要考虑。

#### 预处理与后处理

我们可以研究模型的开始和结束，以获得一些关于其工作原理的见解：

In [4]:
text_en = "Hello World! How's it going?"
resp_fr = translator(text_en)
text_fr = resp_fr[0]['translation_text']

tknzr = translator.tokenizer
tokens_ins = [tknzr.decode(x) for x in tknzr.encode(text_en)]
tokens_in2 = [tknzr.decode(x) for x in translator.preprocess(text_en)['input_ids'][0]]
tokens_out = [tknzr.decode(x) for x in tknzr.encode(text_fr)]
print(f"Inputs Into Preprocessing: {' | '.join(tokens_ins)}")
print(f"Inputs Into Model Forward: {' | '.join(tokens_in2)}")
print(f"Output From Model Forward: {' | '.join(tokens_out)}")

Inputs Into Preprocessing: Hello | World | ! | How | ' | s | it | going | ? | </s>
Inputs Into Model Forward: translate | English | to | French | : | Hello | World | ! | How | ' | s | it | going | ? | </s>
Output From Model Forward: Bonjour | Monde | , | comment | se | passe | - | t | - | il |  | ? | </s>


**通过观察，我们可以看到：**
- 模型对输入和输出语言使用相同的 tokenizer。
- 预处理工作流添加了额外的任务指令以解释任务。

这是因为 T5 模型本身被训练用于多种任务，而英语到法语（en2fr）翻译任务只是其目标之一。定制模型可能会也可能不会训练其直接目标以外的任务，但这样做确实有助于模型学习可转移的直觉。根据训练/架构细节和学习能力：

- **模型可能会超载**，无法适应多任务的形式，并出现退化或过拟合。
- **模型可能会学习到合理的语言先验**，利用任务和指令的共享结构，使进一步的微调更简单、更快速。
- **模型可能会超越其原始训练进行泛化**，具备推理新指令、解决新任务和建立联系的能力。换句话说，展现出像**上下文学习**这样的涌现（emergent）行为。

#### 编码器-解码器前向传递

我们可以通过模型的描述进一步研究这个架构，但您会注意到它们相当冗长。为了方便查看，我们在这里进行了简化：

```python
translator.model           ## See that there's a lot of stuff going on here
translator.model.encoder   ## See that this looks a lot like the BERT model
translator.model.decoder   ## See that this looks roughly the same and wonder what changed
```

<div><img src="imgs/t5-architecture.png" 
     alt="编码器-解码器架构"
     width="1200"/></div>

回顾一下在之前对前向传递的解构中，展示以下属性是相当简单的： 
- 我们可以通过研究模型定义手动指定每个组件的输入。
- 我们可以通过逐个 token 重复前向传递来进行流式生成，将结果积累到缓冲区，并根据需要进行修改/打印。

在这一部分，我们将重复这个过程，但将其模块化为**流生成器**格式，以便于使用。这里的目标是建立一个系统，可以将生成的 tokens 再作为输入，隐藏其中的复杂性，同时允许用户在 tokens 到达时进行迭代和自定义。 

In [5]:
import torch

def get_token_generator(pipeline, model=None, tokenizer=None, max_tokens=50):
    
    ## This method initializes a generator which will yield a stream of tokens
    model = pipeline.model or model
    tknzr = pipeline.tokenizer or tokenizer
    encoder = getattr(model, "encoder", None)
    decoder = ( ## Non-exhaustive resolution
        getattr(model, "decoder", None) 
        or getattr(model, "model", None) 
        or getattr(model, "transformer", None)
    )
    lm_head = getattr(model, "lm_head", None)
    dev = decoder.device
    
    def token_generator(
        encoder_input: str = "",
        decoder_input: str = "",
        max_tokens: int = max_tokens
    ):
        encoder_input_idxs = tknzr.encode(encoder_input)[:-1] * bool(encoder_input)
        decoder_input_idxs = tknzr.encode(decoder_input)[:-1] * bool(decoder_input)
        decoder_inputs = {}

        ## [EncDec] Convert our context into conditioning hidden state for decoder
        if encoder:
            encoder_inputs = {"input_ids": torch.tensor([encoder_input_idxs], device=dev)}
            encoder_outputs = encoder(**encoder_inputs)
            decoder_inputs["encoder_hidden_states"] = encoder_outputs.last_hidden_state
        elif encoder_input_idxs:
            print("`encoder_input` specified despite no encoder being available. Ignoring")
            
        ## [EncDec/Dec] Accumulate decoding starting from <pad> until </s> (eos) is reached.
        buffer_token_idxs = [] if (tknzr.pad_token_id is None) else [tknzr.pad_token_id]
        buffer_token_idxs += decoder_input_idxs
        buffer_token_str = ""
        max_length = len(buffer_token_idxs) + max_tokens
        while len(buffer_token_idxs) < max_length:
            
            ## Pass the current buffer into the decoder, along with encoder states
            ##   NOTE: This one just uses the last hidden state, but some use many more...
            
            decoder_inputs["input_ids"] = torch.tensor([buffer_token_idxs], device=dev).long()
            decoder_outputs = decoder(**decoder_inputs)
            model_outputs = lm_head(decoder_outputs.last_hidden_state)
        
            ## Get the most likely next token and add it to the buffer
            try: sampled_token_idx = torch.argmax(model_outputs, -1)[0][-1].item()
            except: break
            buffer_token_idxs += [sampled_token_idx]
            buffer_token_old = buffer_token_str
            buffer_token_str = tknzr.decode(buffer_token_idxs)
            buffer_token_new = buffer_token_str[len(buffer_token_old):]

            ## Yield (output while keeping spot in the generator call) next token.
            ## If it's end-of-string </s>, break the loop.
            if sampled_token_idx == tknzr.eos_token_id:
                break
            if buffer_token_new:
                yield buffer_token_new
    
    return token_generator

###############################################################################

streamer = get_token_generator(translator)
input_raw_str = "translate English to French: Hello World! How's it going?</s>"

for token in streamer(encoder_input = input_raw_str):
    print(token, end="|")

<pad> Bonjour| Monde|,| comment| se| passe|-|t|-|il| |?|

<br>

## **4.5：创建更通用的模型**

回顾一下我们关于何时使用编码器-解码器的假设，重点关注第 3 和第 4 点。

1. 需要从至少两个可能长度不同的序列中进行推理。
2. 需要逐步生成或继续至少一个序列。
3. **这些序列遵循不相交的分布（格式、目的、模态等）。**
    - 如果它们遵循相同的分布，最好将它们都输入同一个网络路径。
4. **模型需要轻量且任务特定，以便进行训练和/或提高性能。**
    - 如果模型被允许是通用的/多用途，我们可以将两个序列都输入解码器路径。
  
考虑到这一点，让我们引入一个更轻量和通用的编码器-解码器模型：[**谷歌的 Flan-T5 模型类**](https://huggingface.co/docs/transformers/en/model_doc/flan-t5)。

In [6]:
from transformers import pipeline

flan_t5_pipe = pipeline("text2text-generation", model="google/flan-t5-large")

streamer = get_token_generator(flan_t5_pipe)
input_raw_str = "translate English to French: Hello World! How's it going?</s>"

for token in streamer(encoder_input = input_raw_str):
    print(token, end="")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

<pad> Bonjour, monde! Comment se passe-t-il?

基于此，让我们列出可能的选项。假设我们有一个编码器-解码器的结构，前提是 $P$，问题是 $Q$，答案是 $A$。T5 模型理论上可以这样推理：

> `Encoder("{P}: {Q}")` 为 `Decoder("<pad>")` 的生成提供条件，解码器本身被优化为通过逐个生成 token 来产生答案 $A$，直到生成出停止 token。

这种格式通过格式化训练数据来强化，使其符合这一格式，从而形成了强烈的得以理解并遵循这一格式的**归纳偏差**（或 **先验**）。

> <div><img src="imgs/t5-pic.jpg" width="800"/></div>
>
> **来源: [Scaling Instruction-Finetuned Language Models](https://arxiv.org/abs/1910.10683v4)**

此外，Flan 版本的 T5 模型还通过更多复杂任务进行进一步训练，使得模型能够超越其训练目标进行**上下文学习**，也就是仅通过该做什么的上下文，来解决新任务的能力。

> <div><img src="imgs/t5-flan2-spec.jpg" width="1000"/></div>
>
> **来源: [Scaling Instruction-Finetuned Language Models](https://arxiv.org/abs/2210.11416v5)**

为了测试这些能力，我们可以创建一个简单的问题数据集，通过这个**评估过程**测试模型。看看它的表现如何，能否实现最后的练习。

In [7]:
dataset = [
    {   # Simple Translation
        "premise": "Translate from English to Spanish",
        "question": "The book is on the table.",
        "answer": "El libro está en la mesa.",
        "few_shot": {
            "question": "The cat is sleeping on the sofa.",
            "answer": "El gato está durmiendo en el sofá."
    }},{# Commonsense Reasoning
        "premise": "Answer the question using commonsense knowledge",
        "question": "Why can't a fish live out of water?",
        "few_shot": {
            "question": "Why can't humans breathe underwater?",
            "answer": "Humans can't breathe underwater because they need air, not water, to fill their lungs."
    }},{# Creative Story Generation
        "premise": "Continue the story with a creative twist",
        "question": "Once upon a time, in a forest far away, there was a small bear named Timmy who loved honey.",
        "few_shot": {
            "question": "A princess woke up one day to find her castle floating in the sky.",
            "answer": "As she looked outside, she saw a giant eagle carrying the castle on its back, flying towards the mountains."
    }},{  # Mathematical Problem Solving
        "premise": "Solve the mathematical problem",
        "question": "What is the square root of 144?",
        "few_shot": {
            "question": "What is the cube of 3?",
            "answer": "27."
    }},{# Fact-based Question Answering
        "premise": "Answer based on factual knowledge",
        "question": "Who was the first person to walk on the moon?",
        "few_shot": {
            "question": "Who was the first president of the United States?",
            "answer": "The first president of the United States was George Washington."
    }},{# Conversational Continuation
        "premise": "Continue the conversation naturally",
        "question": "User: Can you help me with directions? Agent:",
        "few_shot": {
            "question": "User: What’s the weather like today? Agent:",
            "answer": "It’s sunny and warm with a light breeze."
    }},{# Conversational Continuation
        "premise": "Continue the conversation naturally",
        "question": (
            "User: Can you help me with directions? Agent: Sure, where to and where from?"
            " User: I'd like to get from LA to San Jose today. What's the best road? Agent: "
        ),
        "few_shot": {
            "question": "User: What’s the weather like today? Agent:",
            "answer": "It’s sunny and warm with a light breeze."
    }}
]

streamer = get_token_generator(flan_t5_pipe)

for entry in dataset:
    P, Q = entry['premise'], entry['question']
    FSP, FSQ = entry['few_shot']['question'], entry['few_shot']['answer']
    inputs = {
        "encoder_input": f"{P}: {Q}</s>",
        "decoder_input": "",
        # "encoder_input": f"{P}: ",
        # "decoder_input": f"{FSQ}? {FSA}</s>{Q}? ",
    }
    print(f"{P}: {Q}")
    for token in streamer(**inputs):
        print(token, end="")
    print("\n")

## EXERCISE 1: Incorporate Few-Shot (in this case just one-shot) conditioning.
## EXERCISE 2: Remove the encoder from the equation and progress towards in-context learning.

Translate from English to Spanish: The book is on the table.
<pad> El libro está en la mesa.

Answer the question using commonsense knowledge: Why can't a fish live out of water?
<pad> it is a carnivore

Continue the story with a creative twist: Once upon a time, in a forest far away, there was a small bear named Timmy who loved honey.
<pad> Timmy was a bear who lived in a forest.

Solve the mathematical problem: What is the square root of 144?
<pad> 84

Answer based on factual knowledge: Who was the first person to walk on the moon?
<pad> Neil Armstrong

Continue the conversation naturally: User: Can you help me with directions? Agent:
<pad> Sure, where are you going?

Continue the conversation naturally: User: Can you help me with directions? Agent: Sure, where to and where from? User: I'd like to get from LA to San Jose today. What's the best road? Agent: 
<pad> I can help you with that.



<br>

您会注意到，基础的 Flan-T5 模型在一致性文本生成方面表现尚可，但远达不到开箱即用。**T5 系列真正擅长的是任务特定的微调：**
- 由于模型实际上相当小，因此微调相对简单。
- 由于编码器已经针对多种自然语言进行了微调，它作为基础模型可以在新输入格式上进行推理，只需较少的梯度更新（如果与某个重训练任务共享特征）。
- 由于解码器只是为了生成一个新序列（而不一定是推理来自不同分布的上下文，比如问题和前提这两种不同的分布），它的自回归漂移相对较小，不容易出现偏差。

这种形式下，还有一些不太容易达到的理想特性：
- 通过编码器处理的少样本提示会比较繁琐，需要编码器同时处理输入类和输出类的数据分布。
- 同时，由于这两条路径之间的分叉，解码器总体上可用的训练数据更少。

#### 切换到仅解码器模型

我们之前尝试过 GPT-2 模型，以展示仅解码器模型的功能。效果并不好，这很正常，因为它是一个早期模型，主要适用于微调工作流。那些不易偏移的解码器模型通常需要更大的架构，但一类用来微调的小型仅解码器模型已经出现。这类模型通常被称为 SLM（小型语言模型），并包含了上面 T5 模型的大部分。

练习中，我们将使用微软最小的 Phi SLM 之一，[**Phi-1.5**](https://huggingface.co/microsoft/phi-1_5)。与上面用 Flan-T5 时类似，我们将再次跳过一些必要的定制，以使该系统适用于实际用例，主要是：
- 我们不会进行微调。
- 我们将遵循推荐格式，但不会提前停止/大量提示工程。
- 我们将再次使用一个一年前的开箱即用模型。

In [11]:
decoder_pipe = pipeline("text-generation", model="microsoft/phi-1_5", stop_token="\n", device="cuda")
# decoder_pipe = pipeline("text-generation", model="gpt2", stop_token="\n")
streamer = get_token_generator(decoder_pipe)

for entry in dataset:
    P, Q = entry['premise'], entry['question']
    FSP, FSQ = entry['few_shot']['question'], entry['few_shot']['answer']
    inputs = {
        "decoder_input": f"{P}: {Q}\n\nAnswer: ",
    }
    print("*" * 64)
    for token in streamer(**inputs):
        print(token, end="")
    print("\n")

## EXERCISE: Incorporate Few-Shot (in this case just one-shot) conditioning.

****************************************************************
Translate from English to Spanish: The book is on the table.

Answer: The book is en el table.

Exercise 2:
Translate from French to German: Je m'appelle Marie.

Answer: Je m'appelle Marie.

Exercise 3:
Translate from Mandarin

****************************************************************
Answer the question using commonsense knowledge: Why can't a fish live out of water?

Answer: A fish cannot live out of water because it needs water to breathe and survive.

Exercise 2: Fill in the blank with the correct word: The _ of a car is determined by its engine power.

Answer: The power of

****************************************************************
Continue the story with a creative twist: Once upon a time, in a forest far away, there was a small bear named Timmy who loved honey.

Answer: Once upon a time, in a forest far away, there was a small bear named Timmy who loved honey.

Exercise 2: Rewrite the following sentenc

<br>

如您所见，这个模型有一整套不同的优/缺点。**具体来说，它仅通过单一输入路径就灵活地处理了所有这些新前提，但出现了更多的连贯生成的问题，这跟之前完全无法泛化的情形已大不相同！**

这种**上下文学习**的涌现行为是[**提示工程**](https://en.wikipedia.org/wiki/Prompt_engineering)范式的主要推动力，我们将在第 6 和第 7 个 notebook 中进一步讨论！

<hr>
<br>

# <font color="#76b900">**总结**</font>

目前为止，我们已经看到语言模型如何通过将语言编码作为上下文来生成全新的文本。这为我们打开了许多新可能性，同时也留下了很多未解的问题，但至少我们现在已经处于前沿了，能够在有限的算力下做相当强大的事！接下来，我们将看到一个真正展现编码器-解码器强大的用例；**多模态生成**。

In [ ]:
## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)